In [16]:
import pandas as pd
import numpy as np

all_feature = pd.read_csv('/kaggle/input/abc-feature-labels/all_features(with jerk).csv')
all_label = pd.read_csv('/kaggle/input/abc-feature-labels/all_labels(with jerk).csv')

In [17]:
all_feature = all_feature.to_numpy()
print(all_feature.shape)
print(all_label.shape)
all_label = all_label.to_numpy()
all_label = all_label.T.flatten()
print(all_label.shape)

from sklearn.svm import SVC

(9716, 679)
(9716, 1)
(9716,)


In [18]:
from sklearn.model_selection import RandomizedSearchCV, train_test_split
from sklearn.metrics import classification_report
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler


In [19]:
# Define Support Vector Machine
model = SVC(random_state=42)

# Create Pipeline
steps = list()
steps.append(('scaler', MinMaxScaler()))
steps.append(('classifier', model))
pipeline = Pipeline(steps=steps)

# Define Parameters

params= {
    'classifier__kernel':['linear','poly','rbf','sigmoid'],
    'classifier__C':[10**-2, 10**-1, 10**0, 10**1, 10**2],
    'classifier__class_weight':[None, {0:1,1:5}, {0:1,1:10}, {0:1,1:25}],
    'classifier__decision_function_shape':['ovo', 'ovr'],
    'classifier__gamma':['scale', 'auto']
    
}

#param1 = {}
#param1['classifier__C'] = [10**-2, 10**-1, 10**0, 10**1, 10**2]
#param1['classifier'] = [SVC(random_state=42)]

#param2 = {}
#param2['classifier__class_weight'] = [None, {0:1,1:5}, {0:1,1:10}, {0:1,1:25}]
#param2['classifier'] = [SVC(random_state=42)]

In [20]:
#Random search for ideal reg factor C:
print(f'Searching for the best value of parameter for {model}...')
classifier = RandomizedSearchCV(pipeline, params, cv=5, n_jobs=-1, scoring='f1_macro').fit(all_feature, all_label)
#grid->classifier

# print the best results

#print(f'The best value for parameter n_estimators of SVC is {grid.best_params_}')
#print(f'The highest F1 score is {grid.best_score_}')
# Stats for each test - we have a total 125 tests
classifier.cv_results_#['mean_test_score']
#params_summary = grid.cv_results_['params']
classifier.best_params_
classifier.best_score_


#grid_search_result = result[['param_C','param_kernel','mean_test_score']]
# Capture all data into a Data Frame

#df = pd.DataFrame(list(zip(means, params_summary)), columns=['Mean Score', 'Parameters'])
#df.to_csv(f'GridSearchCV5_n_estimators_SVC.csv', index=False)

Searching for the best value of parameter for SVC(random_state=42)...


0.4943065055172224

In [21]:
print(classifier.cv_results_)
print(classifier.best_params_)
print(classifier.best_score_)
# Create Pipeline

#steps = list()
#steps.append(('scaler', MinMaxScaler()))
#steps.append(('classifier', model))
#pipeline = Pipeline(steps=steps)

# GridSearch for best class_weights:
#print(f'Searching for the best value of parameter max_depth...')
#grid2 = GridSearchCV(pipeline, param2, cv=5, n_jobs=-1, scoring='f1_macro').fit(all_feature, all_label)

# print the best parameters...

#print(f'The best value for parameter max_depth of SVC is {grid2.best_params_}')
# print best score for the best model (in our case roc_auc score)
#print(f'The highest F1 score is {grid2.best_score_}')
# Stats for each test - we have a total 125 tests
#means2 = grid2.cv_results_['mean_test_score']
#params_summary2 = grid2.cv_results_['params']

#Capture all data into a Data Frame

#df2 = pd.DataFrame(list(zip(means2, params_summary2)), columns=['Mean Score', 'Parameters'])
#df2.to_csv(f'GridSearchCV5_class_weight_SVC.csv', index=False)

{'mean_fit_time': array([39.51288037, 81.90899324, 25.48881774, 26.16674023, 30.71709971,
       26.19206691, 57.41518469, 35.71559968, 24.07191243, 44.80194993]), 'std_fit_time': array([4.96986951, 7.06038606, 3.03516856, 2.21624865, 0.75758153,
       0.74459038, 2.76878833, 1.48603122, 1.09352505, 7.03093462]), 'mean_score_time': array([10.40395255,  6.07572803, 14.72861924,  7.06615047, 10.6950388 ,
        9.9145988 , 19.24248219, 17.17579198,  6.94407969,  9.3968225 ]), 'std_score_time': array([1.70104946, 0.89544589, 0.62929818, 0.47182028, 0.3696883 ,
       0.4940008 , 0.41994003, 0.69435036, 0.39054915, 2.35114808]), 'param_classifier__kernel': masked_array(data=['poly', 'linear', 'rbf', 'linear', 'poly', 'linear',
                   'rbf', 'rbf', 'linear', 'poly'],
             mask=[False, False, False, False, False, False, False, False,
                   False, False],
       fill_value='?',
            dtype=object), 'param_classifier__gamma': masked_array(data=['scale',

In [22]:
# Best model parameters are C = 100, class_weights = {0:1,1:5}

#{'classifier__kernel': 'linear', 'classifier__class_weight': None, 'classifier__C': 1}
#0.5088906412845468

model_final = SVC(kernel='linear',gamma='auto',decision_function_shape='ovo', class_weight={0:1,1:5}, C=10,random_state=42)


# Split train-test data
f_train,f_test,l_train,l_test = train_test_split(all_feature, all_label, test_size = 0.2, random_state=42, shuffle = True)

# Scale data
scaler = MinMaxScaler()
scaler.fit(f_train)
f_train = scaler.transform(f_train)
f_test = scaler.transform(f_test)


# Fit + Train data
model_final.fit(f_train,l_train)
prediction = model_final.predict(f_test)

# Evaluate
print(classification_report(l_test,prediction))


              precision    recall  f1-score   support

           0       0.72      0.78      0.75       589
           1       0.40      0.56      0.46        34
           2       0.87      0.81      0.84       171
           3       0.54      0.55      0.55        49
           4       0.57      0.53      0.55       318
           5       0.67      0.55      0.60        53
           6       0.71      0.58      0.64        52
           7       0.87      0.87      0.87       551
           8       0.89      0.81      0.85       127

    accuracy                           0.75      1944
   macro avg       0.69      0.67      0.68      1944
weighted avg       0.75      0.75      0.75      1944

